In [1]:
import pandas as pd
import math
import re
import os

In [2]:
folder = './csv/'

if(not os.path.exists(os.path.join(folder, 'base-cc-serie-historique-2018.csv'))):
    folder = '../csv/'

In [3]:
csv_file = csv_file = os.path.join(folder, 'base-cc-serie-historique-2018.csv')
csv_file

folder += '/filtered'

In [4]:
data = pd.read_csv(csv_file, low_memory=False)
data

,CODGEO,P18_POP,P13_POP,P08_POP,D99_POP,D90_POP,D82_POP,D75_POP,D68_POP,SUPERF,...,D75_LOGVAC,D68_LOGVAC,P18_PMEN,P13_PMEN,P08_PMEN,D99_PMEN,D90_NPER_RP,D82_NPER_RP,D75_NPER_RP,D68_NPER_RP
0,01001,771,767,791,728,579,477,368,347,15.95,...,22.0,25.0,771.000000,767.000000,791.000000,728,579,477.0,368.0,347.0
1,01002,253,236,194,168,159,136,106,88,9.15,...,23.0,1.0,253.000000,236.000000,194.000000,168,159,136.0,106.0,88.0
2,01004,14204,14359,12792,11436,10455,9737,9550,8949,24.60,...,246.0,155.0,13697.547924,13831.019385,12305.616156,11015,10092,9333.0,9164.0,8582.0
3,01005,1720,1635,1567,1408,1156,848,756,627,15.92,...,16.0,15.0,1720.000000,1635.000000,1567.000000,1406,1155,848.0,753.0,627.0
4,01006,112,108,123,86,76,65,82,108,5.88,...,13.0,8.0,112.000000,108.000000,123.000000,86,76,65.0,82.0,108.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34988,97420,23718,22209,22411,18144,14695,13196,11984,11151,57.84,...,267.0,159.0,23614.664615,22128.999131,22367.333836,18135,14685,13154.0,11965.0,11041.0
34989,97421,7224,7226,7294,7402,7004,6467,6462,7105,103.82,...,201.0,138.0,7221.000000,7192.666667,7195.327612,7372,6973,6435.0,6448.0,6881.0
34990,97422,79385,76090,72026,60323,47593,40545,36107,31378,165.43,...,0.0,617.0,78973.785978,75647.548199,71653.651906,60123,47460,39741.0,36001.0,30416.0
34991,97423,7076,7268,6994,6598,5767,5132,4864,4471,42.58,...,113.0,86.0,7070.000283,7260.000367,6990.999857,6597,5763,5132.0,4864.0,4465.0


In [8]:
csv_file = csv_file = os.path.join(folder, 'communes.csv')

data_c = pd.read_csv(csv_file, low_memory=False)
communes = list(data_c["01001"])
communes.append('01001')

data = data[data['CODGEO'].isin(communes)]

data


,CODGEO,P18_POP,P13_POP,P08_POP,D99_POP,D90_POP,D82_POP,D75_POP,D68_POP,SUPERF,...,D75_LOGVAC,D68_LOGVAC,P18_PMEN,P13_PMEN,P08_PMEN,D99_PMEN,D90_NPER_RP,D82_NPER_RP,D75_NPER_RP,D68_NPER_RP
20589,56056,190,172,180,187,192,182,219,271,4.98,...,7.0,7.0,190.0,172.0,180.0,187,192,181.0,219.0,271.0


In [6]:
def getHeaders(dataframe, withRange=False):
    headers = dataframe.columns.tolist()
    ranges = []
    for (index,column) in enumerate(headers):
        tmp = column.split('_')
        if(len(tmp) > 1):
            headers[index] = tmp[1]
        else:
            if(not withRange and re.match('^[a-zA-Z]*\d{4}$', tmp[0])):
                ranges.append(tmp[0][len(tmp[0])-4:])
                tmp[0] = tmp[0][:len(tmp[0]) - 4]

            headers[index] = tmp[0]
    return [headers, ranges]


In [7]:
result = getHeaders(data)
headers = set(result[0])
ranges = set(result[1])
headers


{'CODGEO',
 'DECE',
 'LOG',
 'LOGVAC',
 'NAIS',
 'NPER',
 'PMEN',
 'POP',
 'RP',
 'RSECOCC',
 'SUPERF'}

In [8]:
rheaders = {'DECE': 'Décès',
               'LOG':'Logements',
               'LOGVAC': 'Logements Vacants',
               'NAIS': 'Naissances',
               'NPER':'Nombre personnes par résidence principale',
               'PMEN': 'Personne par ménage',
               'POP': 'Population',
               'RP':'Résidences principales',
               'RSECOCC':'Résidences secondaires',
               'SUPERF':'superficie'
               }
tmp = {}
tmp['code'] = rheaders.keys()
tmp['libelle'] = rheaders.values()


new_csv = os.path.join(folder, f'indicateurs.csv')
pd.DataFrame.from_dict(tmp).to_csv(new_csv, ',', index=False, header=False)



In [9]:
df_1968 = data[["CODGEO", "D68_POP", "D68_LOG", "D68_RP", "D68_RSECOCC",
                "D68_LOGVAC", "D68_NPER_RP", "NAIS6875", "DECE6875", "SUPERF"]]
df_1968


,CODGEO,D68_POP,D68_LOG,D68_RP,D68_RSECOCC,D68_LOGVAC,D68_NPER_RP,NAIS6875,DECE6875,SUPERF
0,01001,347,161.0,119.0,17.0,25.0,347.0,37.0,25.0,15.95
1,01002,88,113.0,40.0,72.0,1.0,88.0,5.0,13.0,9.15
2,01004,8949,3164.0,2891.0,118.0,155.0,8582.0,1220.0,645.0,24.60
3,01005,627,243.0,209.0,19.0,15.0,627.0,99.0,57.0,15.92
4,01006,108,53.0,35.0,10.0,8.0,108.0,5.0,18.0,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,11151,2254.0,2085.0,10.0,159.0,11041.0,2589.0,725.0,57.84
34989,97421,7105,1517.0,1309.0,70.0,138.0,6881.0,1460.0,380.0,103.82
34990,97422,31378,6815.0,5917.0,281.0,617.0,30416.0,6776.0,1454.0,165.43
34991,97423,4471,976.0,853.0,37.0,86.0,4465.0,1070.0,263.0,42.58


In [10]:
df_1975 = data[["CODGEO", "D75_POP", "D75_LOG", "D75_RP",
                "D75_RSECOCC", "D75_LOGVAC", "D75_NPER_RP", "NAIS7582", "DECE7582", "SUPERF"]]
df_1975

,CODGEO,D75_POP,D75_LOG,D75_RP,D75_RSECOCC,D75_LOGVAC,D75_NPER_RP,NAIS7582,DECE7582,SUPERF
0,01001,368,175.0,127.0,26.0,22.0,368.0,28.0,31.0,15.95
1,01002,106,141.0,52.0,66.0,23.0,106.0,4.0,10.0,9.15
2,01004,9550,3601.0,3200.0,155.0,246.0,9164.0,1161.0,607.0,24.60
3,01005,756,283.0,244.0,23.0,16.0,753.0,69.0,44.0,15.92
4,01006,82,55.0,29.0,13.0,13.0,82.0,2.0,14.0,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,11984,2659.0,2383.0,9.0,267.0,11965.0,2438.0,549.0,57.84
34989,97421,6462,1579.0,1287.0,91.0,201.0,6448.0,1197.0,308.0,103.82
34990,97422,36107,8059.0,8059.0,0.0,0.0,36001.0,6644.0,1329.0,165.43
34991,97423,4864,1133.0,969.0,51.0,113.0,4864.0,805.0,208.0,42.58


In [11]:
df_1982 = data[["CODGEO", "D82_POP", "D82_LOG", "D82_RP",
                "D82_RSECOCC", "D82_LOGVAC", "D82_NPER_RP", "NAIS8290", "DECE8290", "SUPERF"]]
df_1982


,CODGEO,D82_POP,D82_LOG,D82_RP,D82_RSECOCC,D82_LOGVAC,D82_NPER_RP,NAIS8290,DECE8290,SUPERF
0,01001,477,208.0,163.0,26.0,19.0,477.0,47.0,30.0,15.95
1,01002,136,139.0,59.0,75.0,5.0,136.0,10.0,10.0,9.15
2,01004,9737,4000.0,3520.0,194.0,286.0,9333.0,1323.0,763.0,24.60
3,01005,848,322.0,275.0,30.0,17.0,848.0,86.0,63.0,15.92
4,01006,65,56.0,27.0,17.0,12.0,65.0,3.0,11.0,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,13196,3124.0,2803.0,36.0,285.0,13154.0,2627.0,596.0,57.84
34989,97421,6467,1798.0,1408.0,73.0,317.0,6435.0,1425.0,387.0,103.82
34990,97422,40545,12030.0,9831.0,563.0,1636.0,39741.0,7781.0,1878.0,165.43
34991,97423,5132,1371.0,1139.0,43.0,189.0,5132.0,977.0,329.0,42.58


In [12]:
df_1990 = data[["CODGEO", "D90_POP", "D90_LOG", "D90_RP", "D90_RSECOCC",
                "D90_LOGVAC", "D90_NPER_RP", "NAIS9099", "DECE9099", "SUPERF"]]
df_1990


,CODGEO,D90_POP,D90_LOG,D90_RP,D90_RSECOCC,D90_LOGVAC,D90_NPER_RP,NAIS9099,DECE9099,SUPERF
0,01001,579,238,196,31,11,579,60.0,45.0,15.95
1,01002,159,142,68,65,9,159,20.0,12.0,9.15
2,01004,10455,4579,4006,174,399,10092,1299.0,835.0,24.60
3,01005,1156,401,369,11,21,1155,135.0,79.0,15.92
4,01006,76,62,33,16,13,76,6.0,14.0,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,14695,3934,3519,15,400,14685,3184.0,776.0,57.84
34989,97421,7004,1990,1663,72,255,6973,1344.0,404.0,103.82
34990,97422,47593,15042,13298,561,1183,47460,9573.0,2454.0,165.43
34991,97423,5767,1646,1416,55,175,5763,1176.0,301.0,42.58


In [13]:
df_1999 = data[["CODGEO", "D99_POP", "D99_LOG", "D99_RP", "D99_RSECOCC",
                "D99_LOGVAC", "D99_PMEN", "NAIS9908", "DECE9908", "SUPERF"]]
df_1999


,CODGEO,D99_POP,D99_LOG,D99_RP,D99_RSECOCC,D99_LOGVAC,D99_PMEN,NAIS9908,DECE9908,SUPERF
0,01001,728,289,248,32,9,728,87.0,49.0,15.95
1,01002,168,142,67,71,4,168,32.0,22.0,9.15
2,01004,11436,5184,4635,135,414,11015,1542.0,855.0,24.60
3,01005,1408,505,473,14,18,1406,159.0,81.0,15.92
4,01006,86,57,41,13,3,86,7.0,8.0,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,18144,5367,5026,31,310,18135,3846.0,912.0,57.84
34989,97421,7402,2397,2055,78,264,7372,1249.0,404.0,103.82
34990,97422,60323,21659,19284,604,1771,60123,10686.0,3064.0,165.43
34991,97423,6598,2068,1838,66,164,6597,1124.0,350.0,42.58


In [14]:
df_2008 = data[["CODGEO", "P08_POP", "P08_LOG", "P08_RP", "P08_RSECOCC",
                "P08_LOGVAC", "P08_PMEN", "NAIS0813", "DECE0813", "SUPERF"]]
df_2008


,CODGEO,P08_POP,P08_LOG,P08_RP,P08_RSECOCC,P08_LOGVAC,P08_PMEN,NAIS0813,DECE0813,SUPERF
0,01001,791,323.530002,291.626387,23.927711,7.975904,791.000000,40,25,15.95
1,01002,194,164.282828,83.282828,61.000000,20.000000,194.000000,15,8,9.15
2,01004,12792,6052.870761,5529.109976,115.156390,408.604395,12305.616156,1067,530,24.60
3,01005,1567,615.187185,586.655322,7.363061,21.168801,1567.000000,114,39,15.92
4,01006,123,68.300000,53.300000,12.000000,3.000000,123.000000,8,4,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,22411,7529.696288,7071.593020,40.265781,417.837486,22367.333836,2031,495,57.84
34989,97421,7294,2695.995271,2272.152712,192.307798,231.534762,7195.327612,653,230,103.82
34990,97422,72026,29733.663409,26046.445868,975.368762,2711.848778,71653.651906,6659,1958,165.43
34991,97423,6994,2632.085127,2283.818340,67.691291,280.575496,6990.999857,565,215,42.58


In [15]:
df_2013 = data[["CODGEO", "P13_POP", "P13_LOG", "P13_RP", "P13_RSECOCC",
                "P13_LOGVAC", "P13_PMEN", "NAIS1318", "DECE1318", "SUPERF"]]
df_2013


,CODGEO,P13_POP,P13_LOG,P13_RP,P13_RSECOCC,P13_LOGVAC,P13_PMEN,NAIS1318,DECE1318,SUPERF
0,01001,767,331.068769,296.966667,21.063063,13.039039,767.000000,44.0,22.0,15.95
1,01002,236,160.597425,101.863248,49.620253,9.113924,236.000000,23.0,5.0,9.15
2,01004,14359,6766.601375,6120.203607,117.640523,528.757245,13831.019385,1092.0,606.0,24.60
3,01005,1635,660.520347,624.107474,9.710099,26.702773,1635.000000,90.0,39.0,15.92
4,01006,108,69.107143,51.107143,13.000000,5.000000,108.000000,7.0,6.0,5.88
...,...,...,...,...,...,...,...,...,...,...
34988,97420,22209,8226.432028,7595.555493,58.482359,572.394176,22128.999131,1844.0,581.0,57.84
34989,97421,7226,2970.565559,2441.612194,162.324313,366.629052,7192.666667,640.0,252.0,103.82
34990,97422,76090,32193.480380,29094.066560,1021.297312,2078.116508,75647.548199,6351.0,2105.0,165.43
34991,97423,7268,2900.463317,2509.428257,65.172510,325.862550,7260.000367,514.0,214.0,42.58


In [16]:
df_2018 = data[["CODGEO", "P18_POP", "P18_LOG", "P18_RP", "P18_RSECOCC", "P18_LOGVAC", "P18_PMEN", "SUPERF"]]
df_2018


,CODGEO,P18_POP,P18_LOG,P18_RP,P18_RSECOCC,P18_LOGVAC,P18_PMEN,SUPERF
0,01001,771,353.730435,311.000000,16.278261,26.452174,771.000000,15.95
1,01002,253,174.392742,105.156379,52.945455,16.290909,253.000000,9.15
2,01004,14204,7341.584896,6531.148636,118.593433,691.842827,13697.547924,24.60
3,01005,1720,773.926652,710.392817,4.632675,58.901159,1720.000000,15.92
4,01006,112,75.546970,53.963636,12.333333,9.250000,112.000000,5.88
...,...,...,...,...,...,...,...,...
34988,97420,23718,9170.187817,8515.285162,42.587533,612.315122,23614.664615,57.84
34989,97421,7224,3237.422083,2591.582784,193.945735,451.893563,7221.000000,103.82
34990,97422,79385,34895.447032,31889.864227,931.563102,2074.019702,78973.785978,165.43
34991,97423,7076,3095.862715,2657.986587,79.262346,358.613783,7070.000283,42.58


In [17]:


dfs = [df_1968,df_1975, df_1982, df_1990, df_1999, df_2008, df_2013,df_2018]

In [18]:
years = {'68': '1968', '75': '1975', '82': '1982', '90': '1990',
         '99': '1999', '08': '2008', '13': '2013', '18': '2018'}

def getRange(str):
    if(not re.match('^[a-zA-Z]*\d{4}$', str)):
        return None
    range = str[len(str)-4:]
    return [years[range[:len(range) - 2]], years[range[len(range) - 2:]]]
    

getRange('DECE0813')

['2008', '2013']

In [19]:
data = {'codegeo':[], 'annee': [], 'indicateur':[], 'valeur':[],'annee_fin': []}
for (index, df) in enumerate(dfs):
    for i, row in df.iterrows():
        [headers, ranges] = getHeaders(df)
        for (j, head) in enumerate(headers):

            if(head != 'CODGEO' and not math.isnan(row[row.keys()[j]])):
                data['annee'].append(list(years.values())[index])
                data['codegeo'].append(row['CODGEO'])
                data['indicateur'].append(head)
                data['valeur'].append(row[row.keys()[j]])

                if head == "DECE" or head == "NAIS":
                    data['annee_fin'].append(getRange(head + ranges[0])[1])
                else:
                    data['annee_fin'].append(None)


pd.DataFrame.from_dict(data).to_csv(os.path.join(
    folder, 'statistiques.csv'), index=False, header=False)
